In [1]:
import pickle
from active_rl.utils.demonstration_memory import RolloutOfflineReplayMemory
from active_rl.utils.demonstration_optimization import standard_optimization
from active_rl.networks.dqn_atari import DQN
import torch.optim as optim 

In [2]:
file_name="data/Breakout/1000.pkl"

In [3]:
memory = RolloutOfflineReplayMemory(file_name, 0.99, 40)

In [4]:
states, actions, rewards, next_states, dones, rollout_rewards, rollout_offset, rollout_next_states, rollout_dones = memory.sample(64)

In [5]:
rewards[20:30]

tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]])

In [6]:
rollout_rewards[20:30]

tensor([[1.6982],
        [1.7326],
        [0.8016],
        [0.6964],
        [1.5358],
        [0.8429],
        [1.6667],
        [0.7250],
        [0.8179],
        [0.9044]])

In [7]:
rollout_offset[20:30]

tensor([[39.],
        [39.],
        [39.],
        [39.],
        [39.],
        [33.],
        [39.],
        [39.],
        [39.],
        [39.]])

In [8]:
device = 'cuda:0'
LR = 0.0000625

In [9]:
policy_net = DQN(4).to(device)
target_net = DQN(4).to(device)
optimizer = optim.Adam(policy_net.parameters(), lr=LR, eps=1.5e-4)

In [10]:
standard_optimization(policy_net, target_net, memory, optimizer, 0.99, 0.2, 0.05, 64, device)

tensor(0.3300, device='cuda:0')

In [11]:
from active_rl.utils.demonstration_training_loop import standard_config, standard_training_loop

In [12]:
standard_config['file_name'] = './data/Breakout'
standard_config['policy_net'] = policy_net
standard_config['target_net'] = target_net
standard_config['optimizer'] = optimizer
standard_config['device'] = device

In [13]:
standard_training_loop(standard_config)

KeyboardInterrupt: 

In [14]:
from active_rl.environments.atari_wrappers import make_atari, wrap_deepmind
from active_rl.utils.atari_utils import fp, ActionSelector, evaluate

In [15]:
env_name = 'Breakout'
env_raw = make_atari('{}NoFrameskip-v4'.format(env_name))
env = wrap_deepmind(env_raw, frame_stack=False, episode_life=True, clip_rewards=True)
c,h,w = c,h,w = fp(env.reset()).shape
n_actions = env.action_space.n

In [17]:
evaluated_reward = evaluate(None, policy_net, device, env_raw, n_actions, eps=0.05, num_episode=15)

In [18]:
evaluated_reward

9.666666666666666

In [19]:
policy_net2 = DQN(4).to(device)

In [20]:
evaluated_reward2 = evaluate(None, policy_net2, device, env_raw, n_actions, eps=0.05, num_episode=15)

In [21]:
evaluated_reward2

1.6